In [46]:
# Import Libraries 

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service


In [47]:
# Parse page 1 of website 

service = Service(executable_path="/Users/jaspreet/Downloads/chromedriver")
driver = webdriver.Chrome(service=service)

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 51) # update this to your liking 


In [48]:
# Store data temporarily then convert to csv 
property_links = []
prices = []
beds = []
baths = []
garages = []
property_type = []
addresses = []
suburbs = []
attributes = []



In [49]:
for page in N_PAGES:
    url = BASE_URL + f"/rent/?ptype=house&sort=dateupdated-desc&state=vic&page={page}"
    driver.get(url) 
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('li', attrs={'class': 'css-1qp9106'}):
        link = element.find('a', href = True)
        property_links.append(link['href'])
        property_type.append('House')
        
        suburb = element.find('span', attrs={"data-testid":"address-line2", 'class' : 'css-iqrvhs'}) # Suburb and postcode only 
        suburbs.append(suburb.text)
        
        address = element.find('a', attrs={"class": "address is-two-lines css-1y2bib4"}) # Full address 
        addresses.append(address.text)
        
        price = element.find('p', attrs={"data-testid":"listing-card-price", 'class': 'css-mgq8yx'})
        prices.append(price.text)
    i = 0
    for element in soup.findAll('div', attrs={"data-testid": "property-features"}):
        attribute = element.find('div', attrs = {'data-testid': 'property-features-wrapper'})
        if attribute: 
            attributes.append(attribute.text)
        else: 
            attributes.append('NULL')
        i = i + 1
    if i == 0: 
        attributes.append('NULL')


In [50]:
# Error in code 
print(len(property_links))
print(len(attributes)) 
attributes.append('NULL')
print(len(attributes)) 


1000
999
1000


In [51]:
for page in N_PAGES:
    url = BASE_URL + f"/rent/?ptype=apartment&sort=dateupdated-desc&state=vic&page={page}"
    driver.get(url) 
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('li', attrs={'class': 'css-1qp9106'}):
        link = element.find('a', href = True)
        property_links.append(link['href'])
        property_type.append('Apartment')
        
        suburb = element.find('span', attrs={"data-testid":"address-line2", 'class' : 'css-iqrvhs'}) # Suburb and postcode only 
        suburbs.append(suburb.text)
        
        full_address = element.find('a', attrs={"class": "address is-two-lines css-1y2bib4"}) # Full address 
        addresses.append(full_address.text)
        
        price = element.find('p', attrs={"data-testid":"listing-card-price", 'class': 'css-mgq8yx'})
        prices.append(price.text)
    
    for element in soup.findAll('div', attrs={"data-testid": "property-features"}):
        attribute = element.find('div', attrs = {'data-testid': 'property-features-wrapper'})
        attributes.append(attribute.text)



In [52]:
attributes.append('NULL')

In [59]:
# Temporarily store data
lists = {'Suburbs': suburbs, 'Addresses': addresses, 'Price per week': prices, 'Property Features': attributes}
df = pd.DataFrame (lists) 
display(df)

,Suburbs,Addresses,Price per week,Property Features
0,BONSHAW VIC 3352,"26 Yolanda Street, BONSHAW VIC 3352",$460 per week ($1999 pcm),4 Beds2 Baths2 Parking
1,KEILOR EAST VIC 3033,"55 Berembong Drive, KEILOR EAST VIC 3033",$425.00,2 Beds1 Bath2 Parking
2,ALFREDTON VIC 3350,"43 Lugano Avenue, ALFREDTON VIC 3350",$420.00 per week,4 Beds2 Baths2 Parking
3,NORTH BENDIGO VIC 3550,"34 St James Wood Drive, NORTH BENDIGO VIC 3550",$460.00 per week,4 Beds2 Baths1 Parking
4,BEVERIDGE VIC 3753,"15 Foundry Circuit, BEVERIDGE VIC 3753",$450 per week ($1955 pcm),4 Beds2 Baths2 Parking
...,...,...,...,...
1995,MANIFOLD HEIGHTS VIC 3218,"3/36 Orr Street, MANIFOLD HEIGHTS VIC 3218",$430 per week,1 Bed1 Bath1 Parking
1996,GLEN IRIS VIC 3146,"310/770B Toorak Road Road, GLEN IRIS VIC 3146",380 Per week,2 Beds1 Bath1 Parking
1997,SOUTH YARRA VIC 3141,"9/63 Millswyn Street, SOUTH YARRA VIC 3141",$495 per week,2 Beds1 Bath1 Parking
1998,FOSTER VIC 3960,"2/1 Victory Avenue, FOSTER VIC 3960",$280,2 Beds1 Bath1 Parking


In [54]:
df.to_csv("Rental_Property_Data.csv")

In [382]:
import re
new_prices = []

weekly_prices = [] 

integer_prices = [] 

for i in prices:

    new_price= re.search(r"\$[^ ]+", i)
    result = new_price.group(0) if new_price else ""
    new_prices.append(result)

for i in new_prices:
    string_split= re.split('pw', i)
    if string_split: 
        weekly_prices.append(string_split[0])
    else: 
        weekly_prices.append('0')



In [384]:
bedrooms = []
bathrooms = [] 
parkings = []
for i in attributes:
    string_split= re.findall('\d+\s*Bed', i)
    if string_split: 
        bedrooms.append(string_split[0])
    else: 
        bedrooms.append('0')

bed_integer = []
for i in bedrooms:
    number = re.split('Bed', i)
    if number:
        bed_integer.append(int(number[0]))
    else: 
        bed_integer.append(int(0))


In [385]:
print(len(bed_integer))

2000


In [386]:
for i in attributes:
    string_split= re.findall('\d+\s*Bath', i)
    if string_split: 
        bathrooms.append(string_split[0])
    else: 
        bathrooms.append('0')
    
bath_integer = []
for i in bathrooms:
    number = re.split('Bath', i)
    if number:
        bath_integer.append(int(number[0]))
    else: 
        bath_integer.append(int(0))


In [387]:
parking_integer = []

In [388]:
for i in attributes:
    string_split= re.findall('\d+\s*Parking', i)
    if string_split:
        parkings.append(string_split[0])
    else: 
        parkings.append('0')
    
parking_integer = []
for i in parkings:
    number = re.split('Parking', i)
    if number:
        parking_integer.append(int(number[0]))
    else: 
        parking_integer.append(int(0))
        

In [389]:
print(len(parking_integer))

2000


In [390]:
land_sizes = []
for i in attributes:
    string_split= re.findall('Parking\d+', i)
    if string_split:
        land_sizes.append(string_split[0])
    else: 
        land_sizes.append('')

In [391]:

new_land_sizes = []
for i in land_sizes:
    land = re.split('Parking', i)
    if land:
        new_land_sizes.append(land[-1])
    else: 
        new_land_sizes.append('')

In [392]:
# Temporarily store data
lists = {'Suburbs': suburbs, 'Addresses': addresses, 'Price per week ($)': weekly_prices, 'Bedrooms': bed_integer, 'Bathrooms': bath_integer, 'Garages': parking_integer, 'Land Sizes (m^2)': new_land_sizes}
df = pd.DataFrame (lists) 
display(df)
df.to_csv("Rental_Property_Data.csv")


,Suburbs,Addresses,Price per week ($),Bedrooms,Bathrooms,Garages,Land Sizes (m^2)
0,BONSHAW VIC 3352,"26 Yolanda Street, BONSHAW VIC 3352",$460,4,2,2,
1,KEILOR EAST VIC 3033,"55 Berembong Drive, KEILOR EAST VIC 3033",$425.00,2,1,2,
2,ALFREDTON VIC 3350,"43 Lugano Avenue, ALFREDTON VIC 3350",$420.00,4,2,2,
3,NORTH BENDIGO VIC 3550,"34 St James Wood Drive, NORTH BENDIGO VIC 3550",$460.00,4,2,1,
4,BEVERIDGE VIC 3753,"15 Foundry Circuit, BEVERIDGE VIC 3753",$450,4,2,2,
...,...,...,...,...,...,...,...
1995,MANIFOLD HEIGHTS VIC 3218,"3/36 Orr Street, MANIFOLD HEIGHTS VIC 3218",$430,1,1,1,
1996,GLEN IRIS VIC 3146,"310/770B Toorak Road Road, GLEN IRIS VIC 3146",,2,1,1,
1997,SOUTH YARRA VIC 3141,"9/63 Millswyn Street, SOUTH YARRA VIC 3141",$495,2,1,1,
1998,FOSTER VIC 3960,"2/1 Victory Avenue, FOSTER VIC 3960",$280,2,1,1,


In [403]:
from bs4 import BeautifulSoup, SoupStrainer
import cchardet as chardet
catchment_lists = [] 

for link in property_links:
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')
    
    schools = []
    for element in soup.findAll('li', attrs={'data-testid': 'fe-co-school-catchment-school'}):
        distance = element.find('div', attrs = {'data-testid': 'fe-co-school-catchment-schoolDistance'})
        schools.append(distance.text)
    
    catchment_lists.append(schools)

KeyboardInterrupt: 

In [405]:
print(len(catchment_lists))

1873


In [406]:
from bs4 import BeautifulSoup, SoupStrainer
import cchardet as chardet
catchment_lists2 = [] 

for link in property_links[1873:]:
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')
    
    schools = []
    for element in soup.findAll('li', attrs={'data-testid': 'fe-co-school-catchment-school'}):
        distance = element.find('div', attrs = {'data-testid': 'fe-co-school-catchment-schoolDistance'})
        schools.append(distance.text)
    
    catchment_lists2.append(schools)

In [407]:
print(len(catchment_lists2))

127


In [408]:
final_list = catchment_lists + catchment_lists2


In [410]:
lists = {'Suburbs': suburbs, 'Addresses': addresses, 'Price per week ($)': weekly_prices, 'Bedrooms': bed_integer, 'Bathrooms': bath_integer, 'Garages': parking_integer, 'Land Sizes (m^2)': new_land_sizes, 'Nearest Schools': final_list}
df = pd.DataFrame (lists) 
display(df)
df.to_csv("Rental_Property_Data.csv")

,Suburbs,Addresses,Price per week ($),Bedrooms,Bathrooms,Garages,Land Sizes (m^2),Nearest Schools
0,BONSHAW VIC 3352,"26 Yolanda Street, BONSHAW VIC 3352",$460,4,2,2,,"[1.4 km away, 1.6 km away, 4.6 km away]"
1,KEILOR EAST VIC 3033,"55 Berembong Drive, KEILOR EAST VIC 3033",$425.00,2,1,2,,"[0.8 km away, 1.3 km away, 4 km away]"
2,ALFREDTON VIC 3350,"43 Lugano Avenue, ALFREDTON VIC 3350",$420.00,4,2,2,,"[1.5 km away, 2.7 km away, 1.5 km away]"
3,NORTH BENDIGO VIC 3550,"34 St James Wood Drive, NORTH BENDIGO VIC 3550",$460.00,4,2,1,,"[1.1 km away, 1.5 km away, 1.9 km away]"
4,BEVERIDGE VIC 3753,"15 Foundry Circuit, BEVERIDGE VIC 3753",$450,4,2,2,,"[2 km away, 5.8 km away]"
...,...,...,...,...,...,...,...,...
1995,MANIFOLD HEIGHTS VIC 3218,"3/36 Orr Street, MANIFOLD HEIGHTS VIC 3218",$430,1,1,1,,"[0.3 km away, 1.2 km away, 2.1 km away]"
1996,GLEN IRIS VIC 3146,"310/770B Toorak Road Road, GLEN IRIS VIC 3146",,2,1,1,,"[0.7 km away, 0.7 km away, 9.2 km away]"
1997,SOUTH YARRA VIC 3141,"9/63 Millswyn Street, SOUTH YARRA VIC 3141",$495,2,1,1,,"[0.8 km away, 2 km away, 2.8 km away]"
1998,FOSTER VIC 3960,"2/1 Victory Avenue, FOSTER VIC 3960",$280,2,1,1,,"[0.2 km away, 0.8 km away]"
